In [ ]:
from __future__ import print_function, division

%matplotlib inline
from matplotlib import pyplot as plt
import seaborn as sns; sns.set(context="poster")
import os
import glob

import numpy as np

from units import M_solar, m_proton, pc, yr, Myr, km, s, gamma

from injection_helpers import get_SNe

from visualize_helpers import \
    total_radial_momentum_of_snapshot, \
    map_to_all_snapshots, \
    get_snapshot_times, \
    load_snapshots
    
from sql_helpers import add_simulation, \
    open_as_DataFrame, get_db_dirname_tmp


In [ ]:
db_dir_tmp = get_db_dirname_tmp()
if not os.path.exists(db_dir_tmp):
    print("making temporary database directory: ", db_dir_tmp)
    os.mkdir(db_dir_tmp)

## 1D (hi-res)

In [ ]:
## Boilerplate path hack to give access to full clustered_SNe package
import sys, os
if __package__ is None:
    if os.pardir not in sys.path[0]:
        file_dir = os.getcwd()
        sys.path.insert(0, os.path.join(file_dir, 
                                        os.pardir, 
                                        os.pardir))
        
from clustered_SNe.analysis.parse import Overview, RunSummary

In [ ]:
data_dir_1D = "1D_data/"

run_summary = RunSummary(data_dir_1D, "25451948-485f-46fe-b87b-f4329d03b203")

# 3D (resolution study)

In [ ]:
base_box_size = 400

N_particles_per_side = np.array([
        100, 
#         150, 
        200, 
#         250, 
        300,
        400, 
        600,
    ])
dxs = base_box_size / N_particles_per_side

simulation_base_name = "cluster_cooling"

run_names = {N : "{}_{}".format(simulation_base_name, N)
             for N in N_particles_per_side}

snapshot_dirs = {N : "../runs/{}/outputs".format(run_names[N])
                for N in run_names}


In [ ]:
SNe_dir = list(snapshot_dirs.values())[0].replace("outputs", "inputs")
SNe = get_SNe(SNe_dir)
SN_times_3D = np.array([SN.time for SN in SNe])

In [ ]:
def copy_and_get_DataFrames(run_name):
    df = open_as_DataFrame(run_name, copy_first=True)
    
    return df


In [ ]:
from multiprocessing import Pool

pool = Pool(3)

# Read in HD runs

In [ ]:
snapshot_dirs_tmp = [snapshot_dirs[N_particles_per_side_i]
                     for N_particles_per_side_i in N_particles_per_side]

run_names_tmp = [run_names[N_particles_per_side_i]
                 for N_particles_per_side_i in N_particles_per_side]


outputs = pool.map(copy_and_get_DataFrames, run_names_tmp)

times_snapshots = {N_particles_per_side_i : outputs[i].time
                   for i, N_particles_per_side_i in enumerate(N_particles_per_side)}

radial_momenta  = {N_particles_per_side_i : outputs[i].momentum
                   for i, N_particles_per_side_i in enumerate(N_particles_per_side)}


# Read in MHD runs

In [ ]:
base_box_size = 400

N_particles_per_side_mhd = np.array([
#         150, 
        200, 
#         250, 
#         300,
#         600,
    ])
dxs_mhd = base_box_size / N_particles_per_side_mhd

simulation_base_name_mhd = "cluster_cooling_mhd"

run_names_mhd = {N : "{}_{}".format(simulation_base_name_mhd, N)
                 for N in N_particles_per_side_mhd}

snapshot_dirs_mhd = {N : "../runs/{}/outputs".format(run_names_mhd[N])
                     for N in N_particles_per_side_mhd}

In [ ]:
snapshot_dirs_tmp = [snapshot_dirs_mhd[N_particles_per_side_i]
                     for N_particles_per_side_i in N_particles_per_side_mhd]

run_names_tmp = [run_names_mhd[N_particles_per_side_i]
                 for N_particles_per_side_i in N_particles_per_side_mhd]

outputs_mhd = pool.map(copy_and_get_DataFrames, run_names_tmp)

times_snapshots_mhd = {N_particles_per_side_i : outputs_mhd[i].time
                       for i, N_particles_per_side_i in enumerate(N_particles_per_side_mhd)}

radial_momenta_mhd = {N_particles_per_side_i : outputs_mhd[i].momentum
                       for i, N_particles_per_side_i in enumerate(N_particles_per_side_mhd)}


# Plot

In [ ]:
def N_particles_to_color(N_particles, verbose=False):
    min_N = min(N_particles_per_side.min(), N_particles_per_side_mhd.min())
    max_N = max(N_particles_per_side.max(), N_particles_per_side_mhd.max())
    
    if verbose:
        print("min_N: ", min_N)
        print("max_N: ", max_N)
    
    # log space the colorscale
    x = (np.log(N_particles)-np.log(min_N)) / (np.log(max_N) - np.log(min_N))
    if verbose:
        print("x:     ", x)
        
    color = plt.cm.viridis_r( x )
    
    return color
    

In [ ]:
with sns.axes_style("ticks"):
    plt.plot((run_summary.times - run_summary.overview.SNe_times[0]) / (Myr),
             run_summary.momentum / (100 * M_solar * 11 * km / s),
             label="1D",
             linestyle="dotted", color="k",
             )


    for N_particles_per_side_i in N_particles_per_side[::-1]:

        plt.plot(times_snapshots[N_particles_per_side_i],
                 radial_momenta[N_particles_per_side_i] / (100 * M_solar * 11 * km / s),
                 label="3D: $\Delta x = $ {:.1f} pc".format(400 / N_particles_per_side_i),
                 color=N_particles_to_color(N_particles_per_side_i),
                 )
        
    for N_particles_per_side_i in N_particles_per_side_mhd[::-1]:

        plt.plot(times_snapshots_mhd[N_particles_per_side_i],
                 radial_momenta_mhd[N_particles_per_side_i] / (100 * M_solar * 11 * km / s),
                 label="3D MHD: $\Delta x = $ {:.1f} pc".format(400 / N_particles_per_side_i),
                 linestyle="dashed", 
                 color=N_particles_to_color(N_particles_per_side_i),
                )


    sns.rugplot(SN_times_3D, color="k", linewidth=3)

    plt.xlabel(r"$t$ $[\mathrm{Myr}]$")
    plt.ylabel(r"$p / (100$ $M_\odot$ $N_\mathrm{SNe})$ $[\mathrm{km}$ $\mathrm{s}^{-1}]$")


    plt.xlim(xmin=-1,
             xmax=40)
    plt.ylim(ymin=300,
             ymax=3000,
            )

    plt.yscale("log")

    plt.legend(loc="best")